In [1]:
import numpy as np
import pandas as pd
import scipy
import glob

In [2]:
pd.options.mode.chained_assignment = None  # default='warn'

In [3]:
from ecoevocrm.consumer_resource_system import *
from ecoevocrm.landscapes import *
import ecoevocrm.utils as utils
import ecoevocrm.viz as viz

In [4]:
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("whitegrid", {'axes.grid' : False})

----------

In [5]:
def alternating_sort(nums):
    # Sort the list in ascending order
    nums_sorted = sorted(nums)
    # Initialize two pointers: one for the smallest, one for the largest
    i, j = 0, len(nums_sorted) - 1
    # Build the result by alternating between smallest and largest
    result = []
    while i <= j:
        result.append(nums_sorted[i])
        i += 1
        if i <= j:
            result.append(nums_sorted[j])
            j -= 1
    return np.array(result)

----------

In [6]:
landscapeAlignmentData_MBE23 = pd.read_csv('/Users/ryan/Dropbox/Projects/Research/kerr/PORTAL/PORTAL/input_data/seed_alignment_scores.csv')
landscapeAlignmentData_MBE23

,Unnamed: 0,seeds,num_of_gen,alignment_perp_dist_all,bin_perp_dist_all,alignment_perp_dist_signeffects,alignment_proportion_signeffects
0,1,1,23,4.968480,4.5,1.712813,0.1500
1,2,2,8,2.177116,2.0,0.570435,0.0875
2,3,3,58,9.863412,9.5,3.481231,0.2375
3,4,4,44,7.250245,7.0,1.988651,0.1625
4,5,5,57,9.385657,9.0,3.718525,0.2625
...,...,...,...,...,...,...,...
9995,9996,9996,55,11.089956,11.0,3.694837,0.2000
9996,9997,9997,49,8.189596,8.0,2.851756,0.2250
9997,9998,9998,13,4.182023,4.0,0.570465,0.0625
9998,9999,9999,10,4.094403,4.0,1.095696,0.0875


In [7]:
outcomeData_MBE23 = pd.read_csv('/Users/ryan/Dropbox/Projects/Research/kerr/PORTAL/PORTAL/input_data/seed_bin_SSWMoutcome.csv')
# outcomeData_MBE23

In [8]:
landscape_dir   = '/Users/ryan/Dropbox/Projects/Research/kerr/PORTAL/PORTAL/input_data/landscapeseeds/'

In [9]:
landscapeEquilibriumTimes_df = pd.read_csv('/Users/ryan/Dropbox/Projects/Research/kerr/PORTAL/PORTAL/results/simgetequilibrium/T2_landscape_equilibrium_times.csv')
# landscapeEquilibriumTimes_df

----------

In [10]:
def scenario_eco_alternating(landscape_seed, plots=False):
    
    chromosomeStr_Apf = '10000'
    chromosomeStr_Apc = '01100'
    chromosomeStr_Bpf = '00001'
    chromosomeStr_Bpc = '00110'
    chromosome_len    = 5
    plasmid_len       = 1+5
    
    landscape_pair_df = pd.read_csv(landscape_dir+str(landscape_seed)+'.csv', dtype={'binary': str})
    landscapeA_df = landscape_pair_df[landscape_pair_df['Species'] == 'Ec']
    landscapeB_df = landscape_pair_df[landscape_pair_df['Species'] == 'Kp']
    landscapeA_df['genotype'] = chromosomeStr_Apc + '1'+landscapeA_df['binary']
    landscapeB_df['genotype'] = chromosomeStr_Bpc + '1'+landscapeB_df['binary']
    landscape_scale_factor = 0.5
    landscapeA_df['cost'] = landscape_scale_factor*(1 - landscapeA_df['mic'])
    landscapeB_df['cost'] = landscape_scale_factor*(1 - landscapeB_df['mic'])
    landscapeA_dict = pd.Series(landscapeA_df['cost'].values, index=landscapeA_df['genotype']).to_dict()
    landscapeB_dict = pd.Series(landscapeB_df['cost'].values, index=landscapeB_df['genotype']).to_dict()
    landscapeA_dict.update({chromosomeStr_Apf+('0'*plasmid_len): landscapeA_df['cost'].max()})
    landscapeB_dict.update({chromosomeStr_Bpf+('0'*plasmid_len): landscapeB_df['cost'].max()})
    landscapeCombined_dict = landscapeA_dict | landscapeB_dict
    
    #~~~~~~~~~~~~~~~~~~~~~~~~
    
    # - - - - - - - - - - - - 
    # Universal parameters:
    # - - - - - - - - - - - - 
    traits_init = np.array([[1, 0, 0, 0, 0,  0,  0, 0, 0, 0, 0],   # pf|A
                            [0, 1, 1, 0, 0,  1,  0, 0, 0, 0, 0],   # pc|A
                            [0, 0, 1, 1, 0,  1,  0, 0, 0, 0, 0],   # pc|B
                            [0, 0, 0, 0, 1,  0,  0, 0, 0, 0, 0]])  # pf|B
    lineageIDs = ['Apf', 'Aanc', 'Banc', 'Bpf']
    m = 1e-9
    mutation_rates = np.array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],   # pf|A 
                               [0, 0, 0, 0, 0, 0, m, m, m, m, m],   # pc|A
                               [0, 0, 0, 0, 0, 0, m, m, m, m, m],   # pc|B
                               [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])  # pf|B
    h = 1
    consumption_rates = np.array([h, h, h, h, h, 0, 0, 0, 0, 0, 0])  # for all types
    l = 0 # 1e-7
    segregation_rates = np.array([0, 0, 0, 0, 0, l, 0, 0, 0, 0, 0])  # for all types
    beta  = 1e-16
    alpha = 1
    transfer_rates_donor = np.array([0, 0, 0, 0, 0, beta,  0, 0, 0, 0, 0])  # for all types
    transfer_rates_recip = np.array([0, 0, 0, 0, 0, alpha, 0, 0, 0, 0, 0])  # for all types
    linkage = {5: [6, 7, 8, 9, 10]}
    segregant_overrides      = { '011000...': {'traits': np.array([1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]), 'mutation_rate': np.array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])},
                                 '001100...': {'traits': np.array([0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0]), 'mutation_rate': np.array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])} }
    transconjugant_overrides = { '100001...': {'traits': {'traits': [0, 1, 2, 3, 4], 'values': np.array([0, 1, 1, 0, 0])}},
                                 '000011...': {'traits': {'traits': [0, 1, 2, 3, 4], 'values': np.array([0, 0, 1, 1, 0])}} }
    cost_baseline = 0.1
    carrying_capacity = 1e9
    
    T_eqAonly = landscapeEquilibriumTimes_df.loc[landscapeEquilibriumTimes_df['landscape_num'] == landscape_seed, 'species_A_fixation_time'].values[0]
    T_phase   = T_eqAonly / 3
    
    # - - - - - - - - - - - - 
    print('No HGT (Species A):')
    # - - - - - - - - - - - - 
    N_init = np.array([1e8, 1e8, 0, 0])
    R_init       = np.array([1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0])
    influx_rates = np.array([1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0])
    community_Aonly = Community(traits=traits_init, cost_landscape=landscapeCombined_dict, cost_baseline=cost_baseline,
                                    consumption_rate=consumption_rates, influx_rate=influx_rates, carrying_capacity=carrying_capacity, 
                                    mutation_rate=mutation_rates, segregation_rate=segregation_rates, transfer_rate_donor=transfer_rates_donor, transfer_rate_recip=transfer_rates_recip,
                                    segregant_overrides=segregant_overrides, transconjugant_overrides=transconjugant_overrides,
                                    segregation_linkage=linkage, transfer_linkage=linkage,
                                    lineageIDs=lineageIDs, lineageID_traits=[6, 7, 8, 9, 10],
                                    N_init=N_init, R_init=R_init, seed=landscape_seed)
    community_Aonly.run(T=T_eqAonly)
    # - - - -
    if(plots):
        fig, ax = plt.subplots(1, 1, figsize=(16, 6))
        viz.abundance_plot(community_Aonly, ax=ax, relative_abundance=False, stacked=True, baseline='sym', log_x_axis=True, log_y_axis=False)
        # fig, ax = plt.subplots(1, 1, figsize=(16, 10))
        # viz.phylogeny_plot(community_Aonly, ax=ax, annot_lineageIDs=True, annot_traits=False, annot_extinct=True, annot_fontsize=7, log_x_axis=True)
        print(community_Aonly.N[community_Aonly.get_extant_type_indices()])
        print(np.array(community_Aonly.type_set.lineageIDs)[community_Aonly.get_extant_type_indices()])
    
    # - - - - - - - - - - - - 
    print('Species A pf pool:')
    # - - - - - - - - - - - - 
    N_init = [1e8, 0, 0, 0]
    R_init       = np.array([1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
    influx_rates = np.array([1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
    community_Apf = Community(traits=traits_init, cost_landscape=landscapeCombined_dict, cost_baseline=cost_baseline,
                                    consumption_rate=consumption_rates, influx_rate=influx_rates, carrying_capacity=carrying_capacity, 
                                    mutation_rate=mutation_rates, segregation_rate=segregation_rates, transfer_rate_donor=transfer_rates_donor, transfer_rate_recip=transfer_rates_recip,
                                    segregant_overrides=segregant_overrides, transconjugant_overrides=transconjugant_overrides,
                                    segregation_linkage=linkage, transfer_linkage=linkage,
                                    lineageIDs=lineageIDs, lineageID_traits=[6, 7, 8, 9, 10],
                                    N_init=N_init, R_init=R_init, seed=landscape_seed)
    community_Apf.run(T=T_phase)
    # - - - -
    if(plots):
        fig, ax = plt.subplots(1, 1, figsize=(16, 6))
        viz.abundance_plot(community_Apf, ax=ax, relative_abundance=False, stacked=True, baseline='sym', log_x_axis=True, log_y_axis=False)
        # fig, ax = plt.subplots(1, 1, figsize=(16, 10))
        # viz.phylogeny_plot(community_Apf, ax=ax, annot_lineageIDs=True, annot_traits=False, annot_extinct=True, annot_fontsize=7, log_x_axis=True)
        print(community_Apf.N[community_Apf.get_extant_type_indices()])
        print(np.array(community_Apf.type_set.lineageIDs)[community_Apf.get_extant_type_indices()])
        
    # - - - - - - - - - - - - 
    print('Species B pf pool:')
    # - - - - - - - - - - - - 
    N_init = np.array([0, 0, 0, 1e8])
    R_init       = np.array([0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0])
    influx_rates = np.array([0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0])
    community_Bpf = Community(traits=traits_init, cost_landscape=landscapeCombined_dict, cost_baseline=cost_baseline,
                                    consumption_rate=consumption_rates, influx_rate=influx_rates, carrying_capacity=carrying_capacity, 
                                    mutation_rate=mutation_rates, segregation_rate=segregation_rates, transfer_rate_donor=transfer_rates_donor, transfer_rate_recip=transfer_rates_recip,
                                    segregant_overrides=segregant_overrides, transconjugant_overrides=transconjugant_overrides,
                                    segregation_linkage=linkage, transfer_linkage=linkage,
                                    lineageIDs=lineageIDs, lineageID_traits=[6, 7, 8, 9, 10],
                                    N_init=N_init, R_init=R_init, seed=landscape_seed)
    community_Bpf.run(T=T_phase)
    # - - - -
    if(plots):
        fig, ax = plt.subplots(1, 1, figsize=(16, 6))
        viz.abundance_plot(community_Bpf, ax=ax, relative_abundance=False, stacked=True, baseline='sym', log_x_axis=True, log_y_axis=False)
        # fig, ax = plt.subplots(1, 1, figsize=(16, 10))
        # viz.phylogeny_plot(community_Bpf, ax=ax, annot_lineageIDs=True, annot_traits=False, annot_extinct=True, annot_fontsize=7, log_x_axis=True)
        print(community_Bpf.N[community_Bpf.get_extant_type_indices()])
        print(np.array(community_Bpf.type_set.lineageIDs)[community_Bpf.get_extant_type_indices()])
        
    # - - - - - - - - - - - - 
    print('HGT Phase 1 (Species A):')
    # - - - - - - - - - - - - 
    N_init = np.array([1e8, 1e8, 0, 0])
    R_init       = np.array([1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0])
    influx_rates = np.array([1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0])
    community_phase1A = Community(traits=traits_init, cost_landscape=landscapeCombined_dict, cost_baseline=cost_baseline,
                                    consumption_rate=consumption_rates, influx_rate=influx_rates, carrying_capacity=carrying_capacity, 
                                    mutation_rate=mutation_rates, segregation_rate=segregation_rates, transfer_rate_donor=transfer_rates_donor, transfer_rate_recip=transfer_rates_recip,
                                    segregant_overrides=segregant_overrides, transconjugant_overrides=transconjugant_overrides,
                                    segregation_linkage=linkage, transfer_linkage=linkage,
                                    lineageIDs=lineageIDs, lineageID_traits=[6, 7, 8, 9, 10],
                                    N_init=N_init, R_init=R_init, seed=landscape_seed)
    community_phase1A.run(T=T_phase)
    # - - - -
    if(plots):
        fig, ax = plt.subplots(1, 1, figsize=(16, 6))
        viz.abundance_plot(community_phase1A, ax=ax, relative_abundance=False, stacked=True, baseline='sym', log_x_axis=True, log_y_axis=False)
        # fig, ax = plt.subplots(1, 1, figsize=(16, 10))
        # viz.phylogeny_plot(community_phase1A, ax=ax, annot_lineageIDs=True, annot_traits=False, annot_extinct=True, annot_fontsize=7, log_x_axis=True)
        print(community_phase1A.N[community_phase1A.get_extant_type_indices()])
        print(np.array(community_phase1A.type_set.lineageIDs)[community_phase1A.get_extant_type_indices()])
        
    # - - - - - - - - - - - - 
    print('HGT Phase 2 (Species B):')
    # - - - - - - - - - - - - 
    community_phase2B = community_Bpf.sample(fraction=1.0)
    community_phase2B.inoculate(community_phase1A.sample(fraction=1.0))
    community_phase2B.run(T=T_phase)
    # - - - -
    if(plots):
        fig, ax = plt.subplots(1, 1, figsize=(16, 6))
        viz.abundance_plot(community_phase2B, ax=ax, relative_abundance=False, stacked=True, baseline='sym', log_x_axis=True, log_y_axis=False)
        # fig, ax = plt.subplots(1, 1, figsize=(16, 10))
        # viz.phylogeny_plot(community_phase2B, ax=ax, annot_lineageIDs=True, annot_traits=False, annot_extinct=True, annot_fontsize=7, log_x_axis=True)
        print(community_phase2B.N[community_phase2B.get_extant_type_indices()])
        print(np.array(community_phase2B.type_set.lineageIDs)[community_phase2B.get_extant_type_indices()])
    
    # - - - - - - - - - - - - 
    print('HGT Phase 3 (Species A):')
    # - - - - - - - - - - - - 
    community_phase3A = community_Apf.sample(fraction=1.0)
    community_phase3A.inoculate(community_phase2B.sample(fraction=1.0))
    community_phase3A.run(T=T_phase)
    # - - - -
    if(plots):
        fig, ax = plt.subplots(1, 1, figsize=(16, 6))
        viz.abundance_plot(community_phase3A, ax=ax, relative_abundance=False, stacked=True, baseline='sym', log_x_axis=True, log_y_axis=False)
        # fig, ax = plt.subplots(1, 1, figsize=(16, 10))
        # viz.phylogeny_plot(community_phase3A, ax=ax, annot_lineageIDs=True, annot_traits=False, annot_extinct=True, annot_fontsize=7, log_x_axis=True)
        print(community_phase3A.N[community_phase3A.get_extant_type_indices()])
        print(np.array(community_phase3A.type_set.lineageIDs)[community_phase3A.get_extant_type_indices()])
    
    # - - - - - - - - - - - - 
    # Compute mean fitnesses:
    # - - - - - - - - - - - - 
    # print()
    # print('*********************')
    # pcTypeIndices_noHGT = [tidx for tidx in range(community_Aonly.num_types) if community_Aonly.type_set.traits[tidx, chromosome_len] == 1]
    pcTypeIndices_noHGT = [tidx for tidx in range(community_Aonly.num_types) if community_Aonly.type_set.trait_keys[tidx][:chromosome_len] == chromosomeStr_Apc]
    meanFitness_noHGT = np.sum((community_Aonly.N[pcTypeIndices_noHGT]/np.sum(community_Aonly.N[pcTypeIndices_noHGT])) * (1 - np.array(community_Aonly.type_set.cost_landscape_bytype)[pcTypeIndices_noHGT]))
    # print(community_Aonly.N[pcTypeIndices_noHGT])
    # print(community_Aonly.N[pcTypeIndices_noHGT]/np.sum(community_Aonly.N[pcTypeIndices_noHGT]))
    # print((1 - np.array(community_Aonly.type_set.cost_landscape_bytype)[pcTypeIndices_noHGT]))
    # print("meanFitness_noHGT", meanFitness_noHGT)
    
    pcTypeIndices_HGT = [tidx for tidx in range(community_phase3A.num_types) if community_phase3A.type_set.trait_keys[tidx][:chromosome_len] == chromosomeStr_Apc]
    meanFitness_HGT = np.sum((community_phase3A.N[pcTypeIndices_HGT]/np.sum(community_phase3A.N[pcTypeIndices_HGT])) * (1 - np.array(community_phase3A.type_set.cost_landscape_bytype)[pcTypeIndices_HGT]))
    # print('---')
    # print(community_phase3A.N[pcTypeIndices_HGT])
    # print(community_phase3A.N[pcTypeIndices_HGT]/np.sum(community_phase3A.N[pcTypeIndices_HGT]))
    # print((1 - np.array(community_phase3A.type_set.cost_landscape_bytype)[pcTypeIndices_HGT]))
    # print("meanFitness_HGT", meanFitness_HGT)
    
    return {'meanFitness_noHGT': meanFitness_noHGT, 'meanFitness_HGT': meanFitness_HGT}


In [11]:
# landscapeBinNums = alternating_sort(landscapeAlignmentData_MBE23['bin_perp_dist_all'].unique())
# numLandscapesPerBin = 10
# num_reps = 10
# for bin in landscapeBinNums:
#     bin_landscapeSeeds = (landscapeAlignmentData_MBE23[landscapeAlignmentData_MBE23['bin_perp_dist_all'] == bin][:]['seeds']).values
#     numSeedsProcessedInThisBin = 0
#     for SEED in bin_landscapeSeeds:
#         outpath = f'./outputs_alteco/L{SEED}.csv'
#         if os.path.isfile(outpath):
#             continue # file already exists
#         data = []
#         for REP in range(num_reps):
#             print(f'\nSEED {SEED}, bin{bin}, REP {REP}\n')
#             # try:
#             results = scenario_eco_alternating(landscape_seed=SEED)
#             data.append({'Treatment': 'T1', 'Replicate': REP, 'Species': 'A', 'Genotype': None, 'Fitness': results['meanFitness_noHGT'], 'LandscapeSeed': SEED, 'LandscapeBin': bin})
#             data.append({'Treatment': 'T3', 'Replicate': REP, 'Species': 'A', 'Genotype': None, 'Fitness': results['meanFitness_HGT'], 'LandscapeSeed': SEED, 'LandscapeBin': bin})
#             # except:
#             #     continue
#         # - - - -
#         # pd.DataFrame(data).to_csv(outpath)
#         # print("\noutput to:", outpath, '\n')
#         # - - - 
#         numSeedsProcessedInThisBin += 1
#         if(numSeedsProcessedInThisBin >= numLandscapesPerBin):
#             break        
#     # break

In [12]:
# stop

In [13]:
# landscapeBinNums = alternating_sort(landscapeAlignmentData_MBE23['bin_perp_dist_all'].unique())
# landscapeBinNums = [0]+list(np.sort(landscapeAlignmentData_MBE23['bin_perp_dist_all'].unique())[::-1])
landscapeBinNums = [12.0, 0.0, 11.0, 1.0, 10.0, 2.0, 9.0, 3.0, 8.0, 4.0, 7.0, 5.0, 6.0,
                    11.5, 0.5, 10.5, 1.5,  9.5, 2.5, 8.5, 3.5, 7.5, 4.5, 6.5, 5.5]
print(landscapeBinNums)
numLandscapesPerBin = 50
num_reps = 100
for bin in landscapeBinNums:
    bin_landscapeSeeds = (landscapeAlignmentData_MBE23[landscapeAlignmentData_MBE23['bin_perp_dist_all'] == bin][:]['seeds']).values
    numSeedsProcessedInThisBin = 0
    print(bin, bin_landscapeSeeds)
    for SEED in bin_landscapeSeeds:
        outpath = f'./outputs_alteco/L{SEED}.csv'
        if os.path.isfile(outpath):
            numSeedsProcessedInThisBin += 1 # optional to count this
            print("ALREADY DONE:", f'SEED {SEED}, bin {bin}')
        elif os.path.isfile(landscape_dir+str(SEED)+'.csv'):
            data = []
            for REP in range(num_reps):
                print("WORKING ON:", f'SEED {SEED}, bin {bin}, REP {REP}')
                # try:
                results = scenario_eco_alternating(landscape_seed=SEED)
                # except ValueError:
                #     continue
                data.append({'Treatment': 'T1', 'Replicate': REP, 'Species': 'A', 'Genotype': None, 'Fitness': results['meanFitness_noHGT'], 'LandscapeSeed': SEED, 'LandscapeBin': bin})
                data.append({'Treatment': 'T3', 'Replicate': REP, 'Species': 'A', 'Genotype': None, 'Fitness': results['meanFitness_HGT'], 'LandscapeSeed': SEED, 'LandscapeBin': bin})
            numSeedsProcessedInThisBin += 1
            pd.DataFrame(data).to_csv(outpath)
            print(">>output to:", outpath)
        else:
            print("LANDSCAPE FILE NOT FOUND:", f'SEED {SEED}, bin {bin}')
        
        # - - - -   
        
        if(numSeedsProcessedInThisBin >= 25): # numLandscapesPerBin):
            break   
            
    # break

[12.0, 0.0, 11.0, 1.0, 10.0, 2.0, 9.0, 3.0, 8.0, 4.0, 7.0, 5.0, 6.0, 11.5, 0.5, 10.5, 1.5, 9.5, 2.5, 8.5, 3.5, 7.5, 4.5, 6.5, 5.5]
12.0 [  45  130  519  848  972 1022 1518 1747 1797 2482 2534 2794 2877 3002
 3039 3203 3337 3397 3801 3964 4120 4193 4218 4239 4257 4313 4408 4448
 4884 4904 4980 5224 5498 5770 5854 5870 6060 6090 6167 6234 6375 6622
 6632 6835 6851 6946 7412 7443 7504 7565 7618 7788 7936 8015 8019 8386
 8515 8871 9438 9639 9871]
WORKING ON: SEED 45, bin 12.0, REP 0
No HGT (Species A):
[ Mutant established at t=56.48 ]	01100100000 --> 01100110000
[ Mutant established at t=57.49 ]	01100100000 --> 01100100100
[ Mutant established at t=58.12 ]	01100100000 --> 01100110000
[ Mutant established at t=70.51 ]	01100100000 --> 01100100010
[ Mutant established at t=126.85 ]	01100100000 --> 01100100010
[ Transconjugant established at t=129.42 ]	01100110000+10000000000 --> 01100110000
[ Mutant established at t=132.87 ]	01100100000 --> 01100100010
[ Mutant established at t=133.46 ]	0110


KeyboardInterrupt

